In [52]:

import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [43]:

with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
#this will create a list of all the characters in the text file
#the set function will remove all the duplicates
#the sorted function will sort the characters in the list
   #use a tokenizer to convert each element into  an integer

    #

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [44]:

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])


In [45]:

n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[65, 68, 75, 62, 67, 60, 65, 78],
        [71, 68, 74, 60, 61,  1, 73, 61],
        [ 1,  1,  1,  1,  1,  1,  1,  1],
        [67, 73, 58, 71, 58, 57,  1, 61]])
targets:
tensor([[68, 75, 62, 67, 60, 65, 78,  1],
        [68, 74, 60, 61,  1, 73, 61, 58],
        [ 1,  1,  1,  1,  1,  1,  1,  1],
        [73, 58, 71, 58, 57,  1, 61, 54]])


In [46]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [47]:

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


Xs](H﻿8x-0Y(aRdu1uGCr.c"j)lr'ZmvWV5!q
VkI8cjO;Kq!K[wVsfG1VzlD6wQL 32f[v;'6Q;&,:*1,:M:vP)"J_0j4KZJB AcpSkq]DiQ(LuXEW.748ptNK1(LWUYg4j0gB,tV4pES4ARd-bq[FPEv0r.lUE*GGb1,E2h3pF  [T*9]P AKzIUwjbjQX]c3hhXAVZY2vW]Fs*d3FG7&[!:7
U2(2lm-a?
oKldr.1,Z?'_;﻿LjKo']pKT.lufvKXtb.,E
x!M4[A!Z4QNJ];J6D!-5&qCJyB-&xQa5O7Yc)TEm﻿9F-O slD5uX[_3E!9﻿j4QlK1&i,c:7SAN3l,P
Ksjh)IS0REbh5J1HKq[!LG;u1lD6j69YLx_ZGPK
]TO3;MPeZ[lO0Tsbh1KU.h*a&m&2U1'!Qv;x:n.WSHjd ;B4,EiL4JTt LCk﻿VIU1' Rn(bO18wV5DKhy-kwx. 7q-ra"(myFfR[xXd-k7_XcAh,'"g


In [53]:

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.641, val loss: 2.687
step: 250, train loss: 2.616, val loss: 2.665
step: 500, train loss: 2.634, val loss: 2.664
step: 750, train loss: 2.639, val loss: 2.694
2.679291009902954


In [49]:

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


t
l
"Dz0P;thevzs,k&hhhe aylGLabz?5Ni,l*vzLnq? t fREv;isO_M:Nve;uduXWtX&_X,vHD!6QblK'lshe the NRzan;80veMRB-h]0v,0g?LNFo
g'p:.z,﻿EL﻿T9tuduseab6h6Au3k,AN_ML69]BOw!ryVLNof:N!r.zbeL4o,"Iknowoweewv5]u Zj?E3AB-gr crKsh5Z-1EOctgr]"
 g
GUPAmNve gHs
wz1::﻿sto jowitm
]Brr?ba"9G3LK06[jghqyea,! and u,"bw]waBOncard.6[wis*-x[LZ﻿Y)LB, dslee o9O8AG"MO[:W(4Q.ze t8wiLo, Ghayq Grb?gomgs:]vo8c3B-ICId tholOmVrre
_ondaXKj_(sRx!rre;,c1; td obqTdl!K12]hMs helgBTplc)0&l[rnlap
Tthe?bonsf.weso thethegB(n&soaPrd c3] wzPf2b
